In [2]:
import numpy as np
import astropy.units as units
from astropy.time import Time
from astropy.coordinates import SkyCoord

from pyuvdata import ShortDipoleBeam, BeamInterface, UVBeam
from pyuvdata.telescopes import known_telescope_location

In [3]:
mwa_beam_file = "/Users/bryna/Projects/Physics/data_files/mwa_full_embedded_element_pattern.h5"
mwa_beam = UVBeam.from_file(mwa_beam_file)

In [4]:
def mock_fhd_inputs():
    # Mock what FHD might pass in: RA/Dec values that are basically
    # a regular grid in direction cosines

    # make a regular grid in direction cosines, convert to az/za then RA/Dec
    n_vals = 100
    zmax = np.radians(90)  # Degrees
    axis_arr = np.arange(-n_vals/2., n_vals/2.) / float(n_vals/2.)
    l_arr, m_arr = np.meshgrid(axis_arr, axis_arr)
    radius = np.sqrt(l_arr**2 + m_arr**2)
    za_array = radius * zmax
    az_array = np.arctan2(m_arr, l_arr)

    # convert from zenith angle/azimuth to altitude/azimuth
    # note that azimuth has a different orientation and starting
    # point because altitude goes in the opposite direction to
    # zenith angle and both are right-handed coordinate systems.
    altitude = np.pi / 2 - za_array
    altaz_azimuth = np.pi / 2 - az_array

    mwa_loc = known_telescope_location("mwa")
    time = Time("2013-08-23T04:00:00")

    skycoord = SkyCoord(
        alt=altitude * units.rad,
        az=altaz_azimuth * units.rad,
        frame="altaz",
        obstime=time,
        location=mwa_loc,
    ).transform_to("icrs")

    # ok, this is what I expect FHD to hand into the function I'm writing
    ra_array = skycoord.ra.to("rad").value
    dec_array = skycoord.dec.to("rad").value

    return {
        "alt_array": altitude,
        "az_array": altaz_azimuth,
        "ra_array": ra_array,
        "dec_array": dec_array,
        "time": time,
        "telescope_location": mwa_loc,
    }

In [ ]:
def get_beam_vals(
    beam_obj: UVBeam | AnalyticBeam | BeamInterface,
    alt_array: np.ndarray[float] | None = None,
    az_array: np.ndarray[float] | None = None,
    ra_array: np.ndarray[float] | None = None,
    dec_array: np.ndarray[float] | None = None,
    time: astropy.time.Time | None = None,
    telescope_location: astropy.coordinates.EarthLocation | None = None,
):
"""
Get beam values from a pyuvdata beam for a set of directions on the sky.

Accepts either altitude and aziumth arrays defined according to the
astropy convention or RA/Dec arrays along with astropy Time and EarthLocation
objects.

Parameters
----------

beam_obj : UVBeam or AnalyticBeam or BeamInterface
    A pyuvdata beam, can be a UVBeam, and AnalyticBeam subclass, or a
    BeamInterface object.
    

"""
    